#NEED TO UPDATE RUSSEL 1000 DATA FRAME SO THAT THE TICKER IS CORRECT, THEN UPDATE ALL FORMULA'S, ADD PEER STOCKS, CHECK NEWS API, FINISH REGRESSION

In [542]:
# Mount to Google Drive

import os, sys
from google.colab import drive
drive.mount('/content/mnt', force_remount=True)
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)

Mounted at /content/mnt


FileExistsError: ignored

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wrds
import datetime
import json

Make function that adds an account with name and preferences, make another function that updates account id with robinhood portolio, make another that adds adds ticker to portfolio, and also one that removes it, then a function that returns the grades for the portfolio as well as a total grade

# First Deliverable

Input robinhood, add a couple extra stocks not in my robinhood portfolio, and 18 weights for each category, to get ESG scores for each of my company using my formulation.

# Second Deliverable

Using my weights, recommend 5 stocks through regression, collecting financial information from the companies

*\
|\
|\
*\
Using Credentials, get robinhood portfolio\
*\
|\
|\
*

In [3]:
!pip3 install robin_stocks
!git clone https://github.com/jmfernandes/robin_stocks.git
!cd robin_stocks
!pip install .

     |████████████████████████████████| 131 kB 36.3 MB/s 
     |████████████████████████████████| 3.5 MB 37.9 MB/s 
Cloning into 'robin_stocks'...
remote: Enumerating objects: 2512, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 2512 (delta 15), reused 8 (delta 3), pack-reused 2484
Receiving objects: 100% (2512/2512), 8.19 MiB | 36.17 MiB/s, done.
Resolving deltas: 100% (1648/1648), done.
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [534]:
from datetime import datetime
from decimal import Decimal
import logging
from logging.handlers import TimedRotatingFileHandler
import pandas as pd
import plotly.express as px 
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import pandas as pd
import datetime as dt
import pandas_datareader.data as web
import plotly.express as px
import plotly.graph_objects as go
import robin_stocks as rh
import json
import datetime

Import Data Frames to Use
* account_df (initialize)
* company_df
* scores_df


In [226]:
account_df = pd.DataFrame()

Function for adding account to accounts_df, using name. Assume default weights (if value does not exist for user, assume preference at 3, else take the value)

In [227]:
def add_to_account(first, last, account_df)-> int:
  df = pd.DataFrame([[first, last, str(pd.to_datetime('now'))]], columns=(['first', 'last', 'date_time']))
  id = df[['first', 'last', 'date_time']].sum(axis=1).map(hash)
  df['id'] = abs(id)
  account_df = account_df.append(df, ignore_index=True)
  return account_df, id

def remove_account(id, account_df):
  account_df_ret = account_df.drop(account_df.index[account_df['id'] == id]).reset_index(drop=True)
  return account_df_ret

In [231]:
# account_df, id = add_to_account("Kachik","Ashkaryan", account_df)
# account_df = remove_account(1719471895094307254, account_df)

Now with account in dataframe, we add function that adjusts the weights.
All categories:
* CG_BD_BF - Board of Directors/Board Functions 
* CG_BD_BS - Board of Directors/Board Structure 
* CG_BD_CP - Board of Directors/Compensation Policy 
* CG_In_VS - Integration/Vision and Strategy 
* CG_Sh_SR - Shareholders /Shareholder Rights 
* Ec_Ma_Pe - Margins /Performance 
* Ec_Pr_SL - Profitability /Shareholder Loyalty 
* Ec_Re_CL - Revenue /Client Loyalty 
* En_En_ER - Emission Reduction 
* En_En_PI - Product Innovation
* En_En_RR - Resource Reduction
* So_Cu_PR - Customer /Product Responsibility 
* So_So_Co - Society /Community 
* So_So_HR - Society /Human Rights 
* So_Wo_DO - Workforce /Diversity and Opportunity 
* So_Wo_EQ - Workforce /Employment Quality 
* So_Wo_HS - Workforce /Health & Safety 
* So_Wo_TD - Workforce /Training and Development 


In [334]:
list_of_categories = \
["CG_BD_BF",
"CG_BD_BS",
"CG_BD_CP",
"CG_In_VS",
"CG_Sh_SR",
"Ec_Ma_Pe",
"Ec_Pr_SL",
"Ec_Re_CL",
"En_En_ER",
"En_En_PI",
"En_En_RR",
"So_Cu_PR",
"So_So_Co",
"So_So_HR",
"So_Wo_DO",
"So_Wo_EQ",
"So_Wo_HS",
"So_Wo_TD"]

In [233]:
def adjust_weight(account_id, category, new_score, account_df):
  list_of_indices = account_df.index[account_df['id'] == account_id]
  account_df.loc[list_of_indices[0], category] = new_score
  return account_df

In [257]:
def get_weight(account_id, category, account_df):
  list_of_index = account_df.index[account_df['id'] == account_id][0]
  try:
    score = account_df.at[list_of_index, category]
  except KeyError:
    return 3.0
  if np.isnan(score):
    return 3.0
  return score

Use config file to input robinhood holding, then logout to ensure data protection

In [278]:
def robinhood_add_portfolio(account_id, config_file_path, account_df):
  config=open(config_file_path).read()
  content=json.loads(config)
  rh.robinhood.authentication.login(content['username'], content['password'])

  portfolio_rh = rh.robinhood.account.build_holdings()
  portfolio_rh_df = pd.DataFrame.from_dict(portfolio_rh, orient='index')
  portfolio_rh_df = portfolio_rh_df.reset_index()
  port_list = portfolio_rh_df['index'].tolist()

  account_df['portfolio'] = account_df['portfolio'].astype('object')

  list_of_index = account_df.index[account_df['id'] == account_id][0]
  account_df.at[list_of_index, 'portfolio'] = port_list
  rh.robinhood.authentication.logout()

  return account_df

In [292]:
def get_portfolio_with_id(account_id, account_df):
  list_of_index = account_df.index[account_df['id'] == account_id][0]
  try:
    total_portfolio = account_df.at[list_of_index, 'portfolio']
  except KeyError:
    return []
  try:
    if np.isnan(total_portfolio):
      return []
  except TypeError:
    return total_portfolio

In [297]:
# get_portfolio_with_id(8222082116939778480, account_df)

[]

In [280]:
# robinhood_add_portfolio(7256861173943290305, '/content/mnt/MyDrive/config.json', account_df)

,first,last,date_time,id,So_Wo_TD,portfolio
0,Kachik,Ashkaryan,2021-11-13 15:40:20.048934,7090982603147374729,5.0,"[NIO, SNDL, LAZR, SOS, WISH, NVDA]"
1,Kachik,Ashkaryan,2021-11-13 15:40:20.544904,4250566806045738170,NaN,NaN
2,Kachik,Ashkaryan,2021-11-13 15:40:21.029351,7256861173943290305,2.0,"[NIO, SNDL, LAZR, SOS, WISH, NVDA]"
3,Kachik,Ashkaryan,2021-11-13 15:40:21.456025,8222082116939778480,NaN,NaN


In [325]:
def add_ticker_to_portfolio(account_id, ticker, account_df):
  list_of_index = account_df.index[account_df['id'] == account_id][0]
  cur_portfolio = get_portfolio_with_id(account_id, account_df)
  if len(cur_portfolio) == 0:
    account_df.at[list_of_index, 'portfolio'] = [ticker]
    return account_df
  else:
    if ticker not in cur_portfolio:
      cur_portfolio.append(ticker) 
      account_df.at[list_of_index, 'portfolio'] = cur_portfolio
    return account_df

def remove_ticker_to_portfolio(account_id, ticker, account_df):
  list_of_index = account_df.index[account_df['id'] == account_id][0]
  cur_portfolio = get_portfolio_with_id(account_id, account_df)
  if ticker in cur_portfolio:
    cur_portfolio.remove(ticker) 
    account_df.at[list_of_index, 'portfolio'] = cur_portfolio
  return account_df

In [ ]:
# robinhood_add_portfolio(7256861173943290305, '/content/mnt/MyDrive/config.json', account_df)

In [351]:
account_df

,first,last,date_time,id,So_Wo_TD,portfolio
0,Kachik,Ashkaryan,2021-11-13 15:40:20.048934,7090982603147374729,5.0,"[TSLA, SNDL]"
1,Kachik,Ashkaryan,2021-11-13 15:40:20.544904,4250566806045738170,NaN,[TSLA]
2,Kachik,Ashkaryan,2021-11-13 15:40:21.029351,7256861173943290305,2.0,"[SNDL, LAZR, SOS, WISH, NVDA, MSFT, NIO]"
3,Kachik,Ashkaryan,2021-11-13 15:40:21.456025,8222082116939778480,NaN,NaN


#NEED TO ADJUST FOR QUERYING DB

---


Make function, given account_id, return the specialized score for each company and total score

* Make function that gets list of weights given account_id
* Make function that gets list of portfolio given account_id
  * def get_portfolio_with_id(account_id, account_df):
* Make function that gives specific score for one company given ticker
* Make function that combines all to give total score, as well as list of scores in same order as portfolio

In [451]:
import json
try:
    # For Python 3.0 and later
        from urllib.request import urlopen
except ImportError:
# Fall back to Python 2's urllib2
    from urllib2 import urlopen

def get_jsonparsed_data(url):
    """
    Receive the content of ``url``, parse it as JSON and return the object.
    """
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)

In [607]:
def get_all_weights(account_id, account_df):
  weights = []
  for category in list_of_categories:
    weights.append(get_weight(account_id, category, account_df))
    
  return weights

def get_personalized_score_for_company(ticker, account_id, account_df, organization_df, esg_df):
  category_weights = get_all_weights(account_id, account_df)

  #normalize account's weights
  normalized = [float(i)/sum(category_weights) for i in category_weights]

  url = f"https://financialmodelingprep.com/api/v3/profile/{ticker}?apikey=813841e1635f4f74e3555996113e7470"
  cusip = -1
  try:
    cusip = get_jsonparsed_data(url)[0]['cusip'][:-1]
  except Exception as e:
    print(e)

  #get specific company's scores
  try:
    org_id = organization_df[organization_df['cusip'] == cusip]['orgid'].values[0]
  except IndexError:
    return None
  esg_tuples = esg_df[esg_df['orgid'] == org_id]

  total_score = 0
  for idx, category in enumerate(list_of_categories):
    total_score += normalized[idx] * np.nanmean(esg_tuples[category.lower()].values)

  return total_score


Function, return total score for user using their portfolio

In [498]:
def get_user_scores_list(account_id, account_df, organization_df, esg_df):
  list_of_comps =  get_portfolio_with_id(account_id, account_df)
  scores_for_each_comp = []
  for company in list_of_comps:
    score = get_personalized_score_for_company(company, account_id, account_df, organization_df, esg_df)
    if score is None:
      print(company)
    else:
      scores_for_each_comp.append((company,score))

  return scores_for_each_comp

In [415]:
comp_has_org_df = pd.read_csv('/content/drive/MyDrive/company_has_org.csv').drop(columns=['Unnamed: 0']).drop_duplicates()
all_ESG_scores = pd.read_csv('/content/drive/MyDrive/orgid_to_allESGcatscores.csv').drop(columns=['Unnamed: 0']).drop_duplicates()

In [500]:
account_df

,first,last,date_time,id,So_Wo_TD,portfolio
0,Kachik,Ashkaryan,2021-11-13 15:40:20.048934,7090982603147374729,5.0,"[TSLA, SNDL]"
1,Kachik,Ashkaryan,2021-11-13 15:40:20.544904,4250566806045738170,NaN,[TSLA]
2,Kachik,Ashkaryan,2021-11-13 15:40:21.029351,7256861173943290305,2.0,"[SNDL, LAZR, SOS, WISH, NVDA, MSFT, NIO]"
3,Kachik,Ashkaryan,2021-11-13 15:40:21.456025,8222082116939778480,NaN,NaN


In [ ]:
# get_personalized_score_for_company('TSLA', 7090982603147374729, account_df, comp_has_org_df, all_ESG_scores)
# get_user_scores_list(7256861173943290305, account_df, comp_has_org_df, all_ESG_scores)

#GET BASIC COMPANY DATA

* Get about information, HQ, news feed, symbol, companyName, industry, website, description, ceo, sector, city, state, image

In [505]:
total_1000_df = pd.read_csv('/content/drive/MyDrive/Stocks in the Russell 1000 Index (1).csv').drop_duplicates()

In [ ]:
total_1000_df

In [ ]:
def company_information_to_display(ticker, total_1000_df):
  spec_comp = total_1000_df[total_1000_df['Symbol'] == ticker]
  company_name = spec_comp['Description']
  industry = spec_comp['GICS Sector']

  url = f"https://financialmodelingprep.com/api/v3/profile/{ticker}?apikey=81c66b2b7a409dde0ec114b052cd0d45"
  try:
    total = get_jsonparsed_data(url)[0]
  except Exception as e:
    print(e)

  headquarters = (total['city'], total['state'])
  website = total['website']
  description = total['description']
  ceo = total['ceo']
  sector = total['sector']
  image = total['image']
  fullTimeEmployees = total['fullTimeEmployees']
  beta = total['beta']

  return spec_comp, company_name, industry, headquarters, website, description,\
  ceo,  sector, image, fullTimeEmployees, beta

company_information_to_display('TSLA', total_1000_df)

In [ ]:
def get_news_feed(ticker):
  url = f"https://financialmodelingprep.com/api/v3/stock_news?tickers={ticker}&limit=5&apikey=a4bedca2df6809daa70d74cf9671699f"
  try:
    total = get_jsonparsed_data(url)[0]
  except Exception as e:
    print(e)

  return total

#GET FINANCE DATA, COMPLETE RECOMMENDATION WITH FILTER AND REGRESSION

real-time stock price (X), daily_increase (dollar and percentage)(X), stock graph, market_cap(X), price-earnings ratio(X), dividend_yield, volume (X), daily_high(X), daily_low(X), open_price, 52_week_high(X), 52_week_low(X), earnings_per_share(X), beta, next_earnings_call, stock_peers

In [ ]:
def get_financial_info(ticker):
  url = f"https://financialmodelingprep.com/api/v3/quote/{ticker}?apikey=a4bedca2df6809daa70d74cf9671699f"
  total=''
  try:
    total = get_jsonparsed_data(url)
  except Exception as e:
    print(e)
  total = total[0]
  current_price = total['price']
  volume = total['volume']

  daily_change_dollar = total['change']
  daily_change_percent = total['changesPercentage']
  daily_low = total['dayLow']
  daily_high = total['dayHigh']
  year_low = total['yearLow']
  year_high = total['yearHigh']
  market_cap = total['marketCap']
  eps = total['eps']
  pe = total['pe']
  earnings_date = total['earningsAnnouncement']
  return current_price, volume, daily_change_dollar, daily_change_percent,daily_low,\
  daily_high, year_low, year_high, market_cap, eps, pe, earnings_date



get_financial_info('AAPL')


###FUNCTION TO GRAPH STOCK

In [ ]:
!pip install yfinance
!pip install cufflinks

In [ ]:
import pandas as pd
import cufflinks as cf
import numpy as np
import yfinance as yf
import warnings
warnings.filterwarnings("ignore")
import plotly.graph_objects as go
from datetime import datetime
import plotly.graph_objects as go
cf.set_config_file(theme='pearl', world_readable=False)
cf.go_offline()

def graph_stock(ticker):
  single_stock = yf.download('msft', start="2020-05-27", end=datetime.today().strftime('%Y-%m-%d'))
  fig = go.Figure( go.Scatter(y=single_stock['Adj Close'], x=single_stock.index ) )
  fig.show(renderer="colab")


In [ ]:
# graph_stock('TSLA')

###FILTER RUSSEL TO TOP 5 BEST IN TERMS OF WEIGHTS

In [603]:
def filter_to_top_10(account_id, account_df, company_df, all_esg, total_1000_df):
  total_list = []
  for comp in total_1000_df['Symbol']:
    score = get_personalized_score_for_company(comp, account_id, account_df, comp_has_org_df, all_ESG_scores)
    total_list.append((comp, score))

  return sorted(total_list, key=lambda x: x[1])[:10]

Variabled needed for regression MV (market-value), CP (cash to price ratio), BP (book-to-price ratio), Var (annualized variance of stock).

Dependent varibale is cumulative excess return (ER)

Regression formula is:\
$ER = α + β_1MV_{it} + β_2Var_{it} + β_3CP_{it} + β_4BP_{it} + β_5ESG_{i, t-1} + ε_{it}$



In [ ]:
def regression_analysis_of_top_10(top_10_list_of_1000):
  pass

In [ ]:
# top_10_list_of_1000 = filter_to_top_10(7090982603147374729, account_df, comp_has_org_df, all_ESG_scores, total_1000_df)